In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import agate

# Census API Key
from config import api_key
c = Census(api_key, year=2009)

In [ ]:
midwest_data = pd.read_csv('MidwestMurderData.csv')
midwest_data

In [ ]:
acs_msa = c.acs5.get(('NAME', 'B01003_001E'), \
                    {'for': 'metropolitan statistical area/micropolitan statistical area:*'})
metrodf = pd.DataFrame(acs_msa)
metrodf = metrodf.rename(columns={"B01003_001E": "Population", "NAME": "MSA1",
                                  "metropolitan statistical area/micropolitan statistical area": "MSA Code"})
metro_only = metrodf[metrodf['MSA1'].str.contains('Metro Area')]
metro_only = metro_only.reset_index(drop=True)

# new data frame with split value columns to isolate 'Metro Area'
new_metro = metro_only["MSA1"].str.split(" Metro Area", n = 1, expand = True)  
# making separate MSA column from new data frame 
metro_only["MSA"]= new_metro[0] 
# Dropping old MSA columns 
metro_only.drop(columns =["MSA1"], inplace = True) 

# new data frame with split columns to isolate state abbreviations
metrostates = metro_only["MSA"].str.split(", ", n = 1, expand = True) 
# add separate state column from new df
metro_only['State'] = metrostates[1]

# create list of unique values in state column to identify all possible midwest state labels in MidwestMurderData.csv
stateslist = metro_only['State'].unique().tolist() 
midweststates = ['OH-PA', 'KS', 'WV-OH', 'WI', 'IA', 'OH', 'IN', 'MO', 'IL', 'IN-MI', 'SD', 'IA-NE-SD', \
                  'MO-IL', 'MO-KS', 'MN', 'MI', 'NE-IA', 'MN-WI', 'KY-IN', 'NE', 'WI-MN', 'WV-KY-OH', \
                  'ND-MN', 'AR-MO', 'IN-KY', 'IA-IL', 'OH-KY-IN', 'IL-IN-WI', 'ND']
# new dataframe with just state values in the midweststates list, reset index 
midwest_metros = metro_only[metro_only['State'].isin(midweststates)]
midwest_metros = midwest_metros.reset_index(drop=True)
midwest_metros

In [ ]:
# census_data_cntyfips = c.acs5.get(('NAME', 'B01003_001E', 'B19301_001E', 'B17001_002E'), {'for': 'county:*', 'in': 'state:*'})
# census_df_cntyfips = pd.DataFrame(census_data_cntyfips)
# census_df_cntyfips = census_df_cntyfips.rename(columns={"B01003_001E": "Population",
#                                                         "B19301_001E": "Per Capita Income",
#                                                         "B17001_002E": "Poverty Count",
#                                                         "NAME": "Name",
#                                                         "county": "County"})
# census_df_cntyfips
